In [ ]:
# Import the libraries
import math
import pandas_datareader as web
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
from datetime import datetime
plt.style.use('fivethirtyeight')


In [ ]:
ticker = input("Enter the Ticker u wish to Predict")

In [ ]:
now = datetime.now() # current date and time
today_date = now.strftime("%Y-%m-%d")

In [ ]:
df = web.DataReader(ticker, data_source='yahoo', start='2012-01-01', end=today_date)
df


In [ ]:
df.shape

In [ ]:

plt.plot(df['Close'])

In [ ]:
df1=df.reset_index()['Close']
df1

In [ ]:
df1.shape

In [ ]:
df1=np.array(df1).reshape(-1,1)

In [ ]:
df1.shape

In [ ]:
df1

In [ ]:
scalar=MinMaxScaler(feature_range=(0,1))
df1=scalar.fit_transform(df1)


In [ ]:
df1

In [ ]:
# split b/w test data and train data
training_size=int(len(df1)*0.65)
test_size=len(df1)-training_size
train_data, test_data=df1[0:training_size,:],df1[training_size:len(df1),:1]



In [ ]:
train_data.shape

In [ ]:
test_data.shape

In [ ]:
# Timeseries data

def create_data (data, time_step=1):
  data1,data2= [], []
  for i in range(len(data)-time_step-1):
    a = data[i:(i+time_step),0]
    data1.append(a)
    data2.append(data[i+time_step,0])
  return np.array(data1), np.array(data2)
  

In [ ]:
time_step=100
x_train, y_train= create_data(train_data,time_step)

In [ ]:
x_train.shape

In [ ]:
y_train

In [ ]:
x_test, y_test =create_data(test_data,time_step)

In [ ]:
x_test.shape

In [ ]:
y_test.shape

In [ ]:
# reshape
x_train=x_train.reshape(x_train.shape[0],x_train.shape[1],1)
x_test=x_test.reshape(x_test.shape[0],x_test.shape[1],1)


In [ ]:
model=Sequential()
model.add(LSTM(50,return_sequences=True,input_shape=(100,1)))
model.add(LSTM(50,return_sequences=True))
model.add(LSTM(50))
model.add(Dense(1))
model.compile(loss='mean_squared_error',optimizer='adam')

In [ ]:
model.summary()

In [ ]:
model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=100,batch_size=64,verbose=1)

In [ ]:
# Let do the prediction and check performance matrics
train_predict=model.predict(x_train)
test_predict=model.predict(x_test)


In [ ]:
train_predict=scalar.inverse_transform(train_predict)
test_predict=scalar.inverse_transform(test_predict)

In [ ]:
#### Calculate RMSE performance metrics
math.sqrt(mean_squared_error(y_train,train_predict))


In [ ]:
### Test Data RMSE
math.sqrt(mean_squared_error(y_test,test_predict))


In [ ]:
## Map
